In [0]:
%pylab inline

In [0]:
import os
import json
import rasterio
import rasterio.features
import shapely.geometry
from affine import Affine

RASTER_FILE = os.path.join(
    os.path.expanduser('~'), 'bh', 'data', 'satellite',
    'SVDNB_npp_20140201-20140228_75N180W_vcmcfg_'
    'v10_c201507201052.avg_rade9.tif'
)
COUNTIES_GEOJSON_FILE = os.path.join(
    os.path.expanduser('~'), 'bh', 'data',
    'us_counties_5m.json'
)
STATES_TEXT_FILE = os.path.join(
    os.path.expanduser('~'), 'bh', 'data',
    'state.txt'
)


The geojson file for us counties is just regular json file. It happens to have
a non-`utf8` encoding (`latin-1`), so to read it we need to specify the encoding
in our call to `json.load`.

We obtained our geojson from [Eric Celeste](http://eric.clst.org/Stuff/USGeoJSON)'s
awesome website. We use the `5m` counties file, which is geojson for U.S. counties
at a resoution of 5 million inches. You can find more information about geojson
and shapefiles
[here](http://chimera.labs.oreilly.com/books/1230000000345/ch12.html#_choose_a_resolution).


In the geojson, the `STATE` property attached to each county is a number rather than a
name. So to convert to readable states, we need to be able to translate these integer codes
to states. It's actually pretty hard to find a reference on what this mapping is, but it turns
out this [census.gov](http://www2.census.gov/geo/docs/reference/state.txt) resource has
everything we need, in a pipe-separated file we can read using `pandas`.


In [0]:
with open(COUNTIES_GEOJSON_FILE, 'r') as f:
    counties_raw_geojson = json.load(f, 'latin-1')

states_df = pd.read_csv(STATES_TEXT_FILE, sep='|').set_index('STATE')
# here we're selecting just the state names column from the indexed DataFrame, so that
# later in the code we can use it as if it were a dict
states = states_df['STATE_NAME']


This top-level geojson object is a `dict` with two keys:
* `type`, which specifies that this is a `FeatureCollection`, and
* `features`, which is a json array of geojson objects for each county.

Since we want to be able to look up counties by name, we'll rearrange this
into a dictionary with county names as keys and the geojson objects for each
county as values, by looking up the `properties.NAME` and `properties.STATE`
key in each county's geojson object (it is important to use the state as well as
name because several states have counties with the same name).

Note that since there are unicode characters in some counties' names, we
have to make sure there's a `u` in front of the formatting string we use,
otherwise we get an ascii error.


In [0]:
def get_county_name_from_geo_obj(geo_obj):
    """
    Use the NAME and STATE properties of a county's geojson
    object to get a name "state: county" for that county.
    """
    return u'{state}: {county}'.format(
        state=states[int(geo_obj['properties']['STATE'])],
        county=geo_obj['properties']['NAME']
    )

counties_geojson = {
    get_county_name_from_geo_obj(county_geojson): county_geojson
    for county_geojson in counties_raw_geojson['features']
}

print sorted(counties_geojson.keys())[:10]


These geojson objects contain a `properties` subobject with metadata
about each county, and a `geometry` object describing the shape of each
county. Python's `shapely` package has tools for working with these.

If you hand the `geometry` dictionary to `shapely.geometry.shape`, you get back
a `MultiPolygon` object that knows how to describe itself. As an extra bonus, it
knows how to display itself in an ipython notebook. Let's take a look at
Manhattan (which is New York county of New York state):


In [0]:
ny_shape = shapely.geometry.shape(counties_geojson['New York: New York']['geometry'])
print '%r' % ny_shape
ny_shape


Now we're ready to combine our county geojson with raster data. We'll use the
`rasterio` package to work with the `.tif` files we pulled from the noaa.gov website.


In [0]:
raster_file = rasterio.open(RASTER_FILE, 'r')


A rasterio file lets you index raster image data as an array of pixels. The
data files are pretty large, but they allow indexing into small slices. Furthermore,
raster files include metadata indicating an affine mapping between pixels and
latitude / longitude coordinates.

Let's try doing this for New York county. We can get the longitude and latitude
bounds of the smallest rectangle containing the whole county by using the
`bounds` property of the `shapely.geometry.MultiPolygon` instance,
which returns `(lon_min, lat_min, lon_max, lat_max)` coordinates:


In [0]:
lon_min, lat_min, lon_max, lat_max = ny_shape.bounds
print lon_min, lat_min, lon_max, lat_max


Next, we can use the `index` method of the rasterio file, which accepts
`(latitude, longitude)` coordinates and returns `(row, col)` indices for
the corresponding pixels. This tells us which part of the raster file we
want to slice into.

There's a strange catch here that can trip you up: in rasterio files,
the first index is for latitude and the second is longitude. So even
though the inputs are `(longitude, latitude)`, the `row` output corresponds
to latitude and the `col` output corresponds to longitude.


In [0]:
bottom, left = raster_file.index(lon_min, lat_min)
top, right = raster_file.index(lon_max, lat_max)

raster_window = ((top, bottom+1), (left, right+1))
raster_window


Note that we used `(top, bottom)` rather than `(bottom, top)`.
The reason, which we will see in more detail later, is that this raster
file orders pixels in reverse order of lattitude.

Now we're ready to load the pixels themselves. To do this we just
pass the window to the `read` method of the raster file, which returns
a numpy float-32 array of pixel intensities. We specify `indexes=1` so
that we'll get a 2d array rather than a 2d array with size 1 - this is
related to more advanced features of the rasterio api that we won't be
using here.


In [0]:
ny_raster_array = raster_file.read(indexes=1, window=raster_window)
ny_raster_array.shape


The `imshow` function plots a matrix as you would write it on the page,
so that the first index runs from top to bottom and the second from left
to right. As a result, we can plot our data as is using imshow, and see
New York's night lights:


In [0]:
from matplotlib import pyplot as plt
plt.imshow(ny_raster_array)
plt.show()


This is pretty cool, but we see a lot of light from outside Manhattan,
because the entire bounding box shows up in our data. How can we
isolate just the data in Manhattan?

For this, we will use some more functionality from `shapely`. First,
thought, we need to adjust the rasterio file's affine mapping to properly map
into the slice of data we are using for our bounding box.

To understand how to do this, we need to think about how rasterio represents
the mapping between latitude and longitude vs pixels. The `affine` property
of the raster file is an `Affine` object:


In [0]:
raster_file.affine


An
```
Affine(a, b, c,
       d, e, f)
```
instance represents a 2d
transformation of the form
$$
\begin{pmatrix} x' \\ y' \\ 1 \end{pmatrix}
=
\begin{pmatrix}
a && b && c \\
d && e && f \\
0 && 0 && 1
\end{pmatrix}
\begin{pmatrix} x \\ y \\ 1 \end{pmatrix}.
$$

In the context of a rasterio file, the original coordinates $x$ and $y$
represent columns and rows for the pixel array, and the affine map $x'$ and $y'$
represent the latitude and longitude.

The `b` and `d` entries are zero because our image is aligned with the equator
and prime meridian. The `a` and `e` coordinates are giving the scalings for
latitude and longitude (the negative `e` means we index top to bottom, as we saw
earlier), while `c` and `f` give the top-left corner (minimum longitude and
maximum latitude) of the image.

When we used `raster_file.read` to get a slice of the full image, we didn't
alter the rotation or scalings, so `a`, `b`, `d` and `e` don't change. But
we need to adjust `c` and `e`, because the top-left corner of our bounding
box for New York county isn't the same as the top-left corner of the full
image:


In [0]:
rfa = raster_file.affine
ny_affine = Affine(
  rfa.a, rfa.b, lon_min,
  rfa.d, rfa.e, lat_max
)

Now that we have an affine mapping for our slice of the data, we can
isolate the pixels that land inside of New York county using
the `rasterio.features.rasterize` function.

This function takes as its first argument an iterable of `(geometry, value)`
pairs. It also takes in an affine mapping from pixel indices to `(longitude,
latitude)` coordinates and an array size, and it returns an array where the
pixel locations inside of each geometry object have been set to the
corresponding value.

By passing our `ny_shape` object with the value 0, and setting the default
`fill` value (which is what `featurize` uses for pixels that aren't in any
of the geometries) to 1, we can obtain a mask array, which is numpy's
convention for working with missing -- or in our case, irrelevant -- data.

Here we create a mask array `mask`, which is 0 for all pixels inside of
New York county and 1 for all pixels outside:



In [0]:
import rasterio.features
ny_mask = rasterio.features.rasterize(
    shapes=[(ny_shape, 0)],
    out_shape=ny_raster_array.shape,
    transform=ny_affine,
    fill=1,
    dtype=np.uint8,
)
plt.imshow(ny_mask)


Finally using this mask we can work with luminosity data for just Manhattan.
For eample, we can re-create our plot of the nighttime lights, zeroing out
all the pixels outside Manhattan:


In [0]:
plt.imshow(ny_raster_array * (1 - ny_mask))


Similarly, by making a `numpy` masked array, we can use numpy's masked-data
functions to compute statistics about the light distribution in new york
county:


In [0]:
ny_masked = np.ma.array(
    data=ny_raster_array,
    mask=ny_mask.astype(bool)
)
print 'min: {}'.format(ny_masked.min())
print 'max: {}'.format(ny_masked.max())
print 'mean: {}'.format(ny_masked.mean())
print 'standard deviation: {}'.format(ny_masked.std())
